In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#追記
import json
import datetime
import math

from urllib import request
from bs4 import BeautifulSoup
import urllib.request
import requests
import re

#importじゃないけど、個人的には省略されて良いことがあまりないので・・
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

In [8]:
df_vr = pd.read_csv("../data/viewing_rating.csv", encoding='cp932')

In [9]:
# time型に
df_vr.start_at=pd.to_datetime(df_vr.start_at)
df_vr["start_at_month"] = df_vr["start_at"].dt.month

In [10]:
df_vr["drama_key"] = df_vr.start_at.dt.strftime("%y")+df_vr.start_at.dt.strftime("%m")+"_"+df_vr.TV_station+"_"+df_vr.time_table

In [11]:
df_vr.drama_key

0       1910_CX_月21
1       1910_CX_火22
2      1910_TBS_火22
3      1910_NTV_水22
4       1910_EX_木22
5       1910_CX_木22
6      1910_NTV_木23
7      1910_TBS_金22
8       1910_EX_金23
9      1910_NTV_土22
10      1910_EX_土23
11     1910_TBS_日21
12     1910_NTV_日22
13      1907_CX_月21
14      1907_CX_火21
15     1907_TBS_火22
16     1907_NTV_水22
17      1907_EX_木21
18      1907_CX_木22
19     1907_NTV_木23
20     1907_TBS_金22
21      1907_EX_金23
22     1907_NTV_土22
23      1907_EX_土23
24     1907_TBS_日21
25     1907_NTV_日22
26      1904_CX_月21
27      1904_CX_火21
28     1904_TBS_火22
29     1904_NTV_水22
30      1904_EX_木21
31      1904_CX_木22
32     1904_NTV_木23
33     1904_TBS_金22
34      1904_EX_金23
35     1904_NTV_土22
36      1904_EX_土23
37     1904_TBS_日21
38     1904_NTV_日22
39      1901_CX_月21
40      1901_CX_火21
41     1901_TBS_火22
42     1901_NTV_水22
43      1901_EX_木21
44      1901_CX_木22
45     1901_NTV_木23
46     1901_TBS_金22
47      1901_EX_金23
48     1901_NTV_土22
49      1901_EX_土23


In [32]:
# URLごとにスクレイピングを行う
def scraping(url):
    
    #ページに複数ある対象部分を取得する
    html = request.urlopen(url)
    soup = BeautifulSoup(html, fromEncoding="utf-8")
    # cast
    tmp_drama_desc_dict = {}
#     for dt in drama_info_tables:
#         cye-lm-tag
    try:
        drama_title = soup.select('h2')[0].select('.bottom')[0]
        print(drama_title.text)
    except:
        print (f"title_error: {url}") 
    #     drama_title = drama_desc[0].select('.cye-lm-tag')
    try:
        drama_desc = soup.select('#exp_det_all')[0] 
        print(drama_desc.text)
    except:
        print (f"desc_error: {url}") 
    
    try:
        drama_info = soup.select_one('#drama_det_all').select("td")
    except:
        print (f"info_error: {url}") 

    
#   breakpoint()
#     print(drama_info)
    drama_info_dict ={}
    tmp_key = ""
    tmp_val = ""
    for  tmp_idx, di in enumerate(drama_info):
#         print(di)
        tmp_key = di.text if tmp_idx % 2 == 0 else tmp_key
        tmp_val = di.text if tmp_idx % 2 == 1 else tmp_val
        try:
            drama_info_dict[tmp_key] = tmp_val
        except KeyError:
            print(f"{tmp_key}_{tmp_val}" )

#     print(drama_info_dict)
    print(drama_info_dict["キー局"])
    print(drama_info_dict["放送曜日"])
    print(drama_info_dict["放送期間"])
    print(drama_info_dict["放送時間"])

    
#     print(drama_info.text)
    
#     breakpoint()
    # <フジテレビ>などを消す
#         [drama_title, TV_station] = reshape_title(drama_title[0].text)
        # キャストスタッフは調整中
#         cast_staff_value = reshape_cast_staff(str(cast_staff[0]))
        
        # 曜日時間
#         weekday_time_value = get_weekday_time(dt)
        # キーの作成 ex) 1710_EX_月21
#         drama_key = start_at + "_" + TV_station + "_" + weekday_time_value
        
#         drama_key = minor_correction(drama_key)
        
#         tmp_drama_info_dict[drama_key] = { "drama_title": drama_title, "TV_station": TV_station, \
#                                           "drama_info" : cast_staff_value}
    
    return tmp_drama_desc_dict

In [33]:
scraping("http://www.tvdrama-db.com/drama_info/p/id-55112")
scraping("http://www.tvdrama-db.com/drama_info/p/id-60465")



ショムニ2013

										今から10年前、会社から何の役にも立たないゴミ同然の存在だと烙印を押された社員が送り込まれた部署があった…。その名は「庶務二課」。しかし、その薄暗い庶務二課の部屋で、その生活を楽しむ者たちが現れた。彼女たちは欲望のままに行動し、結果的に会社を救い退職していった。そして「ショムニ」は廃止され、坪井千夏（江角マキコ）は消息不明に…。それから10年後…。日本は長い不況に陥りリストラの嵐が吹き荒れる。もちろん、満帆商事にも…。オリンピックを期待される棒高跳びの選手だった円山詩織（本田翼）は、広告塔として入社したが、ケガで選手生命を絶たれ、陸上部も廃止になり社内で邪魔扱いされ、悪口のツイッター攻撃も受けていた…。社員食堂にいても詩織には誰も近づかない。そんな中で、彼女にストレートな行動を取るOLたちが現れる。ネットに精通した頭脳明晰女子・三波まどか（ベッキー）、気配を感じさせない不気味女子・安倍麗子（安藤サクラ）、フェロモン全開お色気女子・小島美鈴（森カンナ）、そして健康オタクのおしゃべり女子・福田益代（堀内敬子）だ。ある日、落ち込む詩織は1人の女性に出会う。誰かに追われている彼女は、突然詩織の財布からタクシー代として1万円を抜き取り、去っていった…。翌日、出社した詩織は同じ女性の姿を目撃する。彼女こそ「伝説の女」坪井千夏だった。いったいなぜ、彼女は再び満帆商事に姿を現したのか!?【以上、フジテレビ広報資料より引用】協力：バスク、フジアール、ベイシス。車輌：ファン、コマツサポートサービス。提供：花王、ホンダ、アイフル、アコム、モビット、小林製薬(1)-(3)、YKK AP(1)-(3)、GALAXY S4(4)-(10)、サントリー(4)-(7)、今夜アナタと眠りたい(8)-(10)。特別協力：東京湾クルーズ・レストランシップ(9)。【データ協力：LED】
				
CX
水
2013/07/10～2013/09/18
22:00-22:54
家売るオンナ

										2017年度「東京ドラマアウォード」連続ドラマ部門優秀賞受賞作品。「天才的不動産屋・三軒家万智。彼女に売れない家はないという。客の個人的な問題に首を突っ込み、人生最大の買い物を予測不能な方法で次々と売っていく、爽快お仕事ドラマ。中堅不動産会社・テーコー不動産。売

{}

In [35]:
###########
# 実行本体
#
drama_desc_dict = {}
# for tmp_did in range(30000, 67000):
for tmp_did in range(60000, 60100):
    
    tmp_url = "http://www.tvdrama-db.com/drama_info/p/id-" + str(tmp_did)
    print (tmp_url)
    tmp_drama_desc_dict = scraping(tmp_url)
#     drama_desc_dict.update(tmp_drama_desc_dict)

import pprint
pprint.pprint(drama_desc_dict)


http://www.tvdrama-db.com/drama_info/p/id-60000
てれび絵本「しごとばシリーズ　客室乗務員（「客室乗務員」にルビ「きゃくしつじょうむいん」が付く）」

										こどもたちに人気の「しごとばシリーズ」、今回は、私たちが飛行機の機内で快適な時間を過ごせるように、空の旅のお手伝いをしてくれる『客室乗務員』の仕事を紹介。【以上、NHK広報資料より引用】
				
NHK ETV
火
2016/02/02～2016/02/02
08:55-09:00
http://www.tvdrama-db.com/drama_info/p/id-60001
てれび絵本「しごとばシリーズ　新幹線運転士」

										こどもたちに人気の「しごとばシリーズ」、今回は男の子が大好きな乗り物、新幹線。それを運転する『新幹線運転士』の登場です。新幹線を走らせるための色々な作業を詳しく紹介。【以上、NHK広報資料より引用】
				
NHK ETV
火
2015/07/20～2015/07/20
08:55-09:00
http://www.tvdrama-db.com/drama_info/p/id-60002
てれび絵本「しごとばシリーズ　すし職人・歯医者」

										こどもたちに人気の「しごとばシリーズ」、今回はみんなが大好きなおいしいお寿司をつくる『すし職人』、そして痛くなった歯を治療してくれる『歯医者』紹介。【以上、NHK広報資料より引用】
				
NHK ETV
火
2015/07/21～2015/07/21
08:55-09:00
http://www.tvdrama-db.com/drama_info/p/id-60003
てれび絵本「しごとばシリーズ　パティシエ・美容師」

										こどもたちに人気の「しごとばシリーズ」、今回はおいしいケーキをつくる女の子に人気の職業『パティシエ』、そしていろいろなヘアスタイルで美しく変身させてくれる『美容師』を紹介。【以上、NHK広報資料より引用】
				
NHK ETV
水
2015/07/22～2015/07/22
08:55-09:00
http://www.tvdrama-db.com/drama_info/p/id-60004
てれび絵本「しご

ラヴソング（誤り…ラブソング）

										福山雅治演じる“元プロミュージシャン”神代広平（かみしろ・こうへい）が、人と話すのが苦手だが、美しい歌声を持つ女性と出会い、音楽を通して心を通わせ、失いかけた人生を取り戻す物語。【以上、フジテレビ広報資料より引用】第１回は拡大版（月曜21:00～22:24）。【データ協力：パパン】
				
CX
月
2016/04/11～2016/06/13
21:00-21:54
http://www.tvdrama-db.com/drama_info/p/id-60019
黒い樹海

										両親を早くに亡くした笠原祥子（北川景子）は、たったひとりの姉・信子（小池栄子）と都内のマンションで２人暮らし。仕事を探しながら、新聞社に勤務する信子を信頼し、寄り添うように生きてきた。その日も東北旅行に出発する信子と別れ、祥子は何社目かの面接に出かけた。それはいつもと変わらぬ、仲のよい姉妹の光景のはずだった。ところがその夜、信子が事故で亡くなったという知らせが入る。しかも、どういうわけか、祥子に告げていた旅先とは異なる場所で、命を落としてしまったのだ…。悲しみにくれる祥子の前に現れたのは、信子の同僚記者・吉井亮一（向井理）。何を考えているのか、信子の死の謎について祥子と共に調査に乗り出す吉井。その一方で、祥子は姉の後釜として新聞社の文化部で働きだし、生前、姉が担当していた文化人たちに会いに行く。彼らは皆、華やかな表の顔からは想像もつかないほどクセのある人物で、中でも小児科医・西脇満太郎（沢村一樹）はセレブ界きっての遊び人らしく、何度も祥子に誘いをかけてくる始末だった。ところが直後、姉の同僚だった町田知枝（酒井若菜）が自殺。知枝は信子の死について「責任をとるべき人がいる」「有名人ならなおさらよ」と謎めいた言葉を祥子に残していた。姉の死に、信子が生前つき合っていた著名人の誰かが関わっていることを直感した祥子。だが、真相を知ろうとすればするほど、渦中の人物が次々と殺害されていく事態が発生！　祥子は図らずも “黒い樹海”のような深い闇に吸い込まれていく…。【以上、テレビ朝日広報資料より引用】協力：エス・エヌ企画、北九州市立松本清張記念館、日本文学振興会松本清張賞事務局。撮影協力：長野県大町市、長野県大町市産業観光部観光課、国土交通

警視庁捜査一課９係（警視庁捜査一課９係 season11）

										第１回は拡大版スペシャルのため、別データとして収録している。
				
EX
水
2016/04/06～2016/06/15
21:00-21:54
http://www.tvdrama-db.com/drama_info/p/id-60038
山本周五郎人情時代劇（第九話）しじみ河岸

										深川の貧しい長屋で、寝たきりの父・勝次（中原和宏）と盲目の弟・直次郎（中島来星）を養う器量良しの娘・お絹（柊子）が恋人の卯之吉（奥村秀人）殺しで捕まる。吟味与力・高木新左衛門（山崎画大）は、短刀で刺したという自供を受け、お絹が下手人だと断定。しかし、牢でお絹を見た新参者の吟味与力・花房律之助（松尾敏伸）は異議を唱え、再吟味を始める。律之助は短刀が特徴的であることから、これを作った職人から依頼主を捜そうと刀工屋へ。その後、長屋界隈や現場となったしじみ河岸で聞き込みに回るが、皆刺々しい雰囲気で目撃者の長屋の差配人・源兵衛（長谷川公彦）をはじめ、勝次、直次郎も何も語らない。そんな中、質屋・両替商「相模屋」が勝次と直次郎を援助していた上、短刀の依頼主だったことが判明。だが、「相模屋」の若旦那・清太郎（タモト清嵐）は、短刀は以前なくしたと説明する。正しい裁きを望む律之助は、詮議所でお絹に真相を聞き出そうとするが、お絹は自分が下手人で死罪も受け入れると言い張る。貧苦な環境が関係していると睨んだ律之助は、卯之吉殺し申し渡しの前日にお絹と対峙し、勝次と直次郎が長屋を追い出されることになったと告げ、最後の賭けに出るが…。【以上、BSジャパン広報資料より引用】撮影協力：EDO WONDERLAND Studio。車輌：虫狩　愉司（制作応援も）。協力会社：山田かつら、ビッグウッド、キアロスクーロ撮影事務所、日本シネ・サービス、SOTO、アイズ、ノマド、OXYBOT。提供：NECネッツエスアイ、全農、大和証券グループ。【データ協力：LED】
				
BSジャパン
火
2016/02/02～2016/02/02
21:00-21:54
http://www.tvdrama-db.com/drama_info/p/id-60039
山本周五郎人情時代劇（第十話）泥棒と若殿

										武家の次男・

心のメッセージ　「二本のローソク」

										中学校道徳・特別活動向け教育番組で、ドラマ仕立てのフォーマットとされるため当データベースに収録している。
				
NHK ETV
金
1988/06/03～1988/06/03
13:10-13:30
http://www.tvdrama-db.com/drama_info/p/id-60072
心のメッセージ　「ぼくと一緒に行く人は」

										中学校道徳・特別活動向け教育番組で、ドラマ仕立てのフォーマットとされるため当データベースに収録している。
				
NHK ETV
金
1988/09/02～1988/09/02
13:10-13:30
http://www.tvdrama-db.com/drama_info/p/id-60073
心のメッセージ　「藍を愛す」

										中学校道徳・特別活動向け教育番組で、ドラマ仕立てのフォーマットとされるため当データベースに収録している。
				
NHK ETV
金
1988/09/30～1988/09/30
13:10-13:30
http://www.tvdrama-db.com/drama_info/p/id-60074
心のメッセージ　「近所のあいつ」

										中学校道徳・特別活動向け教育番組で、ドラマ仕立てのフォーマットとされるため当データベースに収録している。
				
NHK ETV
金
1988/10/14～1988/10/14
13:10-13:30
http://www.tvdrama-db.com/drama_info/p/id-60075
心のメッセージ　「守れ！湿原・霧多布」

										中学校道徳・特別活動向け教育番組で、ドラマ仕立てのフォーマットとされるため当データベースに収録している。
				
NHK ETV
金
1988/10/28～1988/10/28
13:10-13:30
http://www.tvdrama-db.com/drama_info/p/id-60076
心のメッセージ　「健康はやる気から」

										中学校道徳・特別活動向け教育番組で、ドラマ仕立てのフォーマットとされるため当データベースに収録している。
				
NHK ETV
金
1989/0

喜びも悲しみも

										詳細不明。「日本映画女優全集」（キネマ旬報社刊）の「岸田今日子」の項目に本作に関する記述がある。
				
CX

～

http://www.tvdrama-db.com/drama_info/p/id-60092
日本のヴァイオリン王～名古屋が生んだ世界のマエストロ　鈴木政吉物語～

										三味線職人の政吉（武田真治）がヴァイオリンを作り出したのは明治時代半ばのこと。知り合いが持っていたヴァイオリンの音色に興味を抱いたのが、この楽器との出会いだった。政吉はすぐに見よう見まねでヴァイオリンを完成させるも、構造がデタラメだと指摘を受けてしまう。職人魂に火がついた政吉は寝食を忘れ、ヴァイオリン作りに没頭。やがて、政吉のヴァイオリンは世間に知られるようになり、ぽつぽつと注文が入るようになる。その後、政吉は妻・乃婦（笛木優子）と馴染みの芸者・良（中村ゆり）、さらにそれぞれの間に出来た子供たちを同居させるなど、破天荒なふるまいを見せながら、ヴァイオリン作りに邁進する。自作のヴァイオリンが国内外で高評価を得るようなった政吉は、ヴァイオリンを機械で作ることを目指す。誰でも気軽にヴァイオリンに触れられる時代を作ると意気込む政吉だったが、齢60を過ぎた頃、自分のヴァイオリンとの向き合い方や、これまでの人生が否定されるような出来事が起きてしまう。【以上、東海テレビ広報資料より引用】資料提供：「日本のヴァイオリン王」（中央公論新社）井上さつき著、「鈴木鎮一と共に」（主婦の友社）、「幸田姉妹」（ショパン）萩谷由喜子著、松下　敏幸（弦楽器製作者　マエストロ）、クヮルテット・ハウス・ジャパン、朝日新聞社、毎日新聞社。協力：鈴木バイオリン製造、公益社団法人才能教育研究会、鈴木鎮一記念館、宗次ホール、愛知県立芸術大学、ヤマハ、島村楽器、日本近代音楽館、東京藝術大学（東京芸術大学）、日本弦楽器指導者協会、日本弦楽器製作者協会、安藤　　毅、菊地　俊一、谷口　　勤、鈴木　節子、福島　淑子、松浦　正義、山崎美術（クレジット表示では「崎」は「たつさき（立崎）」）、おかもと技粧、コンガス、東海テレビプロダクション、名古屋東通、東海ネクスト、スクラッチ、シナリオプリント。写真提供：中央区立京橋図書館、横浜市中央図書館、PIXTA、イタリア政府観光局、Foto

In [16]:
import json
fw = open('../data/drama_desc0122.json','w')
# json.dump関数でファイルに書き込む
json.dump(simple_json ,fw,indent=4, ensure_ascii=False)


NameError: name 'simple_json' is not defined